# Face Reconstruction


## Tested models

### Basic

In [1]:
from models.basic.Discriminator import Discriminator as BasicDiscriminator
from models.basic.Generator import Generator as BasicGenerator
from models.basic_2.Discriminator import  Discriminator as BasicDiscriminator_2
from models.basic_2.Generator import Generator as BasicGenerator_2
from models.AOT.Discriminator import Discriminator as Discriminator_AOT
from models.AOT.Generator import Generator as BasicGenerator_AOT
from models.double_gan.UNet import UNet as DoubleDiscriminator_UNet
from models.double_gan.UNet_maskAware import UNet_maskAware as DoubleGenerator_UNet
from models.double_gan.Generator import Generator as DoubleGenerator_AOT
from classes.Experiment import Experiment
from train import train
from config import n_epochs


In [3]:
Experiments = [  Experiment(name = 'Overfit_Study_Baseline',
                              generator_models = [BasicGenerator_2],
                              discriminator_models = [DoubleDiscriminator_UNet],
                              generate_whole_image= False,
                              overfittingStudy=True,
                              pretrain = True,
                              dropout_generator = 0.01,
                              dropout_discriminator = 0.01,
                              epochs = n_epochs),
                Experiment(name = 'Overfit_Study_DoubleUnet',
                              generator_models = [DoubleGenerator_UNet],
                              discriminator_models = [DoubleDiscriminator_UNet],
                              generate_whole_image= True,
                              overfittingStudy=True,
                              pretrain = True,
                              dropout_generator = 0.01,
                              dropout_discriminator = 0.01,
                              epochs = n_epochs),
                Experiment(name = 'Overfit_Study_AOT',
                              generator_models = [BasicGenerator_AOT],
                              discriminator_models = [DoubleDiscriminator_UNet],
                              generate_whole_image= True,
                              overfittingStudy=True,
                              pretrain = True,
                              dropout_generator = 0.1,
                              dropout_discriminator = 0.1,
                              epochs = n_epochs)]

In [4]:
from train import train

In [5]:
for experiment in Experiments:
    train(experiment)

https://app.neptune.ai/zukowskanina1/FaceReconstruction/e/FAC-111
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


c:\Users\ismyn\Anaconda3\envs\FaceReconstruction\lib\site-packages\neptune\new\attributes\attribute.py:64: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` instead.
  return self.assign(value, wait)


INITIALISING WEIGHTS


Training Epoch 171 :  23%|██▎       | 3/13 [00:04<00:11,  1.11s/it, disc_loss=0.244, gen_adv_loss=0.309, gen_pixel_loss=0.219]